In [5]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector

from datetime import datetime

In [39]:
#Funcion para insertar un coche en la bbdd
def insertar_coche(df_coches);

#Conexion a la base de datos de MySQL
conn = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "Whereisbotus1996",
    database = "coches_segunda_mano"
)

columnas_coches = ["marca", "modelo", "localizacion", "tipo_combustible", "kilometros", "tipo_carroceria", "tipo_cambio" , "potencia_cv", "garantia", "distintivo_ambiental", "precio", "peso_masa_kg", "puertas", "plazas", "cilindros", "co2", "deposito", "month", "year", "fecha_extraccion"]

cursor = conn.cursor()
    
    for index, row in df_coches.iterrows():
        coche_data = {
            'marca': row['marca'],
            'modelo': row['modelo'],
            'localizacion': row['localizacion'],
            'tipo_combustible': row['tipo_combustible'],
            'kilometros': row['kilometros'],
            'tipo_carroceria': row['tipo_carroceria'],
            'tipo_cambio': row['tipo_combustible'],
            'potencia_cv': row['potencia_cv'],
            'garantia': row['garantia'],
            'distintivo_ambiental': row['distintivo_ambiental'],
            'precio': row['precio'],
            'peso_masa_kg': row['peso_masa_kg'],
            'puertas': row['puertas'],
            'plazas': row['plazas'],
            'cilindros': row['cilindros'],
            'co2': row['co2'],
            'deposito': row['deposito'],
            'month': row['month'],
            'year': row['year'],
            'fecha_extraccion': datetime.now() 
        }
        
        valores = (
            coche_data['marca'],
            coche_data['localizacion'],
            coche_data['año'],
            coche_data['tipo_combustible'],
            coche_data['kilometros'],
            coche_data['tipo_carroceria'],
            coche_data['tipo_cambio'],
            coche_data['potencia_cv'],
            coche_data['garantia'],
            coche_data['distintivo_ambiental'],
            coche_data['precio'],
            coche_data['peso_masa_kg'],
            coche_data['puertas'],
            coche_data['plazas'],
            coche_data['cilindros'],
            coche_data['co2'],
            coche_data['deposito'],
            coche_data['month'],
            coche_data['year'],
            coche_data['fecha_extraccion']
            
        query = f"""
            INSERT INTO coches_segunda_mano ({columnas_coches})
            VALUES ({', '.join(['%s'] * len(valores))});
        """
            
        cursor.execute(query, valores)
    
    conn.commit()
    cursor.close()
    conn.close()
    print("datos insertador correctamente")        

SyntaxError: expected ':' (2273797638.py, line 2)

In [ ]:
#Probamos la funcion
insertar_datos_coches(df_coches)

In [32]:
#Cerrar la conexion al final del proceso
cursor.close()
conn.close()